<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-24 12:11:25
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.40 C
-------------------
Today PnL: -1.47 L (-1.04%)
Current PnL: -34.46 L (-21.54%)
CY Booked + Current PnL: -18.78 L (-11.74%)
-------------------
Total profit:  1.24 L
Total loss:  -35.70 L
-------------------
Total Booked + Current PnL: 8.20 L (6.16%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.2%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.01 C (72.21%)
Deployed:  1.33 C
Current:  1.40 C
CAGR/XIRR %: 2.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.23,3.15,17.85,21.57,18041.0,3090.0,101070.0,397.04,-11.02,57.0,H-SC,6.18,167.0,0.17,0.73,47.49,XR,ATH,FINANCE
18,COALINDIA,0.13,4.89,12.18,17.67,19223.0,7364.0,157824.0,478.51,26.26,56.0,L-LC,11.54,185.0,0.38,1.15,26.69,XY25,ATH,MINING
77,TTKPRESTIG,0.79,-30.59,44.19,0.09,30913.0,-30823.0,69954.0,770.00,62.43,32.0,M-SC,10.49,250.0,-1.00,0.51,0.79,OX40N,NTT,DURABLES
51,MEDANTA,-1.72,-4.31,29.67,24.08,36042.0,-5474.0,121476.0,1486.00,-5.04,51.0,X-SC,9.36,87.0,-0.15,0.88,11.26,XY24,NTT,HEALTHCARE
36,ICICIGI,-2.39,1.02,18.13,19.34,36650.0,2044.0,202153.0,2252.93,-14.93,50.0,X-MC,1.74,66.0,0.06,1.47,17.12,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,1.98,-14.81,125.87,92.42,298691.0,-41258.0,237301.0,400.0,103.30,74.0,H-SC,6.49,174.0,-0.14,1.73,32.05,XY24,NTT,BANKS
41,INDUSINDBK,1.17,-31.30,93.44,32.89,49559.0,-24168.0,53038.0,1800.0,-350.00,56.0,L-MC,11.51,258.0,-0.49,0.39,46.06,XR,NTT,BANKS
0,5PAISA,0.94,-38.51,83.62,12.90,112341.0,-84149.0,134347.0,593.0,98.32,43.0,H-SC,7.43,170.0,-0.75,0.98,11.71,OX40N,NTT,FINANCE
49,LAOPALA,0.85,-36.55,119.20,39.09,99162.0,-47915.0,83190.0,464.0,81.55,65.0,H-SC,13.67,154.0,-0.48,0.60,11.25,AR,NTT,CERAMICS
77,TTKPRESTIG,0.79,-30.59,44.19,0.09,30913.0,-30823.0,69954.0,770.0,62.43,32.0,M-SC,10.49,250.0,-1.00,0.51,0.79,OX40N,NTT,DURABLES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,-5.66,-12.68,41.82,23.83,88342.0,-30680.0,211244.0,1908.19,-8.19,18.0,X-LC,3.86,8.0,-0.35,1.54,1.35,X40,ATH,IT
72,TATAELXSI,-4.99,-35.45,104.53,32.03,88956.0,-46734.0,85101.0,9161.00,-19.19,26.0,H-SC,9.35,158.0,-0.53,0.62,0.00,OX40N,NTT,IT
25,EASEMYTRIP,-4.81,-51.99,203.45,45.70,172719.0,-91920.0,84895.0,26.40,-11.99,63.0,M-SC,18.85,204.0,-0.53,0.62,39.87,XY24,NTT,TRAVEL
71,TANLA,-4.37,-50.02,339.60,119.69,459523.0,-135448.0,135313.0,1943.92,-49.49,35.0,H-SC,19.10,147.0,-0.29,0.98,6.90,AR,ATH,IT
42,INFY,-4.31,-13.52,55.24,34.24,149465.0,-42314.0,270574.0,1972.00,-31.79,17.0,X-LC,6.33,3.0,-0.28,1.97,0.00,X40,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-2.39,2.01,103.81,107.91,156358.0,2975.0,150619.0,1641.55,-8.17,56.0,M-SC,7.95,220.0,0.02,1.10,13.32,OX40N,ATH,CABLES
77,TTKPRESTIG,0.79,-30.59,44.19,0.09,30913.0,-30823.0,69954.0,770.00,62.43,32.0,M-SC,10.49,250.0,-1.00,0.51,0.79,OX40N,NTT,DURABLES
66,SIS,-0.37,-31.41,76.80,21.26,58488.0,-34876.0,76156.0,528.00,1787.05,32.0,H-SC,4.43,168.0,-0.60,0.55,4.99,OX40N,NTT,MISC
40,INDIGOPNTS,-0.53,-32.81,48.85,0.02,57297.0,-57267.0,117292.0,1408.00,79.13,28.0,M-SC,10.84,222.0,-1.00,0.85,2.02,OX40N,NTT,PAINTS
48,KANSAINER,0.48,-33.37,70.31,13.47,126330.0,-89991.0,179676.0,340.00,-42.69,28.0,H-SC,11.18,162.0,-0.71,1.31,0.48,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,0.59,21.05,19.95,45.2,46303.0,40353.0,232095.0,1856.0,38.62,64.0,X-MC,7.4,79.0,0.87,1.69,29.27,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.13,4.89,12.18,17.67,19223.0,7364.0,157824.0,478.51,26.26,56.0,L-LC,11.54,185.0,0.38,1.15,26.69,XY25,ATH,MINING
26,FINCABLES,-2.39,2.01,103.81,107.91,156358.0,2975.0,150619.0,1641.55,-8.17,56.0,M-SC,7.95,220.0,0.02,1.10,13.32,OX40N,ATH,CABLES
50,MASFIN,-1.23,3.15,17.85,21.57,18041.0,3090.0,101070.0,397.04,-11.02,57.0,H-SC,6.18,167.0,0.17,0.73,47.49,XR,ATH,FINANCE
1,ABB,0.00,8.57,34.08,45.57,96801.0,22421.0,284040.0,7934.00,-30.22,64.0,H-MC,7.62,125.0,0.23,2.07,26.13,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,0.00,8.57,34.08,45.57,96801.0,22421.0,284040.0,7934.00,-30.22,64.0,H-MC,7.62,125.0,0.23,2.07,26.13,AR,NTT,ELECTRICAL
50,MASFIN,-1.23,3.15,17.85,21.57,18041.0,3090.0,101070.0,397.04,-11.02,57.0,H-SC,6.18,167.0,0.17,0.73,47.49,XR,ATH,FINANCE
26,FINCABLES,-2.39,2.01,103.81,107.91,156358.0,2975.0,150619.0,1641.55,-8.17,56.0,M-SC,7.95,220.0,0.02,1.10,13.32,OX40N,ATH,CABLES
39,INDIAMART,-0.78,-5.46,118.65,106.72,138352.0,-6731.0,116605.0,4810.62,-57.42,48.0,H-SC,7.97,140.0,-0.05,0.85,16.65,AR,ATH,MISC
85,ZYDUSLIFE,-0.29,-4.53,42.07,35.63,84170.0,-9503.0,200071.0,1286.17,-17.12,51.0,H-MC,5.09,120.0,-0.11,1.45,11.38,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-4.31,-13.52,55.24,34.24,149465.0,-42314.0,270574.0,1972.00,-31.79,17.0,X-LC,6.33,3.0,-0.28,1.97,0.00,X40,NTT,IT
33,HCLTECH,-5.66,-12.68,41.82,23.83,88342.0,-30680.0,211244.0,1908.19,-8.19,18.0,X-LC,3.86,8.0,-0.35,1.54,1.35,X40,ATH,IT
8,AWL,-1.90,-37.79,152.67,57.19,309754.0,-123236.0,202891.0,485.00,-64.48,20.0,X-SC,17.89,81.0,-0.40,1.48,0.00,XY24,NTT,FMCG
73,TCS,-0.37,-23.29,61.10,23.58,202767.0,-100768.0,331861.0,4311.59,-33.46,27.0,X-LC,6.17,2.0,-0.50,2.41,0.00,X40,ATH,IT
12,BATAINDIA,-1.15,-48.20,161.02,35.20,107318.0,-62021.0,66649.0,2096.00,-12.64,30.0,X-SC,21.87,91.0,-0.58,0.48,0.00,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,-0.25,3.83,16.32,20.77,39675.0,8961.0,243105.0,1644.00,-7.86,57.0,X-MC,0.56,64.0,0.23,1.77,11.51,X40N,NTT,BREWERIES
21,DABUR,-0.08,0.48,43.01,43.70,107209.0,1198.0,249266.0,735.00,-8.76,52.0,X-MC,1.35,70.0,0.01,1.81,15.61,XY24,BTT,FMCG
36,ICICIGI,-2.39,1.02,18.13,19.34,36650.0,2044.0,202153.0,2252.93,-14.93,50.0,X-MC,1.74,66.0,0.06,1.47,17.12,X40,ATH,INSURANCE
65,SIEMENS,0.05,-11.24,41.40,25.51,68387.0,-20910.0,165185.0,4671.50,38.78,62.0,H-LC,2.45,51.0,-0.31,1.20,21.73,AR,ATH,ELECTRICAL
34,HINDUNILVR,0.41,-8.39,24.08,13.67,56708.0,-21563.0,235500.0,2922.00,-32.06,49.0,X-LC,2.76,9.0,-0.38,1.71,11.18,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
2,ABBOTINDIA,-0.74,-11.86,33.95,18.07,53522.0,-21205.0,157650.0,35195.0,-25.06,37.0,X-MC,11.21,58.0,-0.40,1.15,0.0,X40,ATH,PHARMA
12,BATAINDIA,-1.15,-48.20,161.02,35.20,107318.0,-62021.0,66649.0,2096.0,-12.64,30.0,X-SC,21.87,91.0,-0.58,0.48,0.0,X40,NTT,FOOTWEAR
9,BAJAJHFL,-0.18,-25.39,108.38,55.47,196727.0,-61770.0,181516.0,181.5,-36.04,31.0,X-MC,17.72,60.0,-0.31,1.32,0.0,X40N,ATH,FINANCE
8,AWL,-1.90,-37.79,152.67,57.19,309754.0,-123236.0,202891.0,485.0,-64.48,20.0,X-SC,17.89,81.0,-0.40,1.48,0.0,XY24,NTT,FMCG
58,RELAXO,-1.57,-54.52,234.85,52.31,155062.0,-79134.0,66026.0,1176.0,-51.60,36.0,X-SC,14.04,92.0,-0.51,0.48,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-1.70,-32.58,111.86,42.84,49026.0,-21178.0,43828.0,424.00,-55.92,43.0,X-SC,17.90,80.0,-0.43,0.32,5.43,XY24,NTT,MISC
12,BATAINDIA,-1.15,-48.20,161.02,35.20,107318.0,-62021.0,66649.0,2096.00,-12.64,30.0,X-SC,21.87,91.0,-0.58,0.48,0.00,X40,NTT,FOOTWEAR
58,RELAXO,-1.57,-54.52,234.85,52.31,155062.0,-79134.0,66026.0,1176.00,-51.60,36.0,X-SC,14.04,92.0,-0.51,0.48,0.00,X40N,NTT,FOOTWEAR
51,MEDANTA,-1.72,-4.31,29.67,24.08,36042.0,-5474.0,121476.0,1486.00,-5.04,51.0,X-SC,9.36,87.0,-0.15,0.88,11.26,XY24,NTT,HEALTHCARE
35,HONAUT,-0.16,-22.59,85.14,43.32,107345.0,-36792.0,126080.0,58357.33,-32.00,43.0,X-SC,11.39,90.0,-0.34,0.92,2.37,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.78,-14.96,40.00,19.06,115710.0,-50892.0,289274.0,452.00,-53.50,47.0,X-LC,17.77,1.0,-0.44,2.10,4.74,X40,NTT,FMCG
73,TCS,-0.37,-23.29,61.10,23.58,202767.0,-100768.0,331861.0,4311.59,-33.46,27.0,X-LC,6.17,2.0,-0.50,2.41,0.00,X40,ATH,IT
42,INFY,-4.31,-13.52,55.24,34.24,149465.0,-42314.0,270574.0,1972.00,-31.79,17.0,X-LC,6.33,3.0,-0.28,1.97,0.00,X40,NTT,IT
75,TMPV,-0.42,-15.92,58.58,33.33,160465.0,-51875.0,273925.0,600.00,-82.53,58.0,X-LC,3.00,4.0,-0.32,1.99,11.99,XY24,NTT,AUTO
33,HCLTECH,-5.66,-12.68,41.82,23.83,88342.0,-30680.0,211244.0,1908.19,-8.19,18.0,X-LC,3.86,8.0,-0.35,1.54,1.35,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1.17,-31.30,93.44,32.89,49559.0,-24168.0,53038.0,1800.00,-350.00,56.0,L-MC,11.51,258.0,-0.49,0.39,46.06,XR,NTT,BANKS
67,SONACOMS,-1.49,-8.65,52.26,39.09,48293.0,-8750.0,92409.0,804.02,-27.63,59.0,M-MC,9.56,193.0,-0.18,0.67,30.73,AR,ATH,AUTO
50,MASFIN,-1.23,3.15,17.85,21.57,18041.0,3090.0,101070.0,397.04,-11.02,57.0,H-SC,6.18,167.0,0.17,0.73,47.49,XR,ATH,FINANCE
18,COALINDIA,0.13,4.89,12.18,17.67,19223.0,7364.0,157824.0,478.51,26.26,56.0,L-LC,11.54,185.0,0.38,1.15,26.69,XY25,ATH,MINING
37,ICICIPRULI,-0.32,9.55,19.34,30.73,41738.0,18807.0,215812.0,790.00,-19.89,58.0,X-MC,4.06,78.0,0.45,1.57,23.53,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-4.81,-51.99,203.45,45.70,172719.0,-91920.0,84895.0,26.4,-11.99,63.0,M-SC,18.85,204.0,-0.53,0.62,39.87,XY24,NTT,TRAVEL
1,ABB,0.00,8.57,34.08,45.57,96801.0,22421.0,284040.0,7934.0,-30.22,64.0,H-MC,7.62,125.0,0.23,2.07,26.13,AR,NTT,ELECTRICAL
7,ATULAUTO,-1.46,-18.36,78.30,45.57,126015.0,-36193.0,160939.0,844.0,3483.78,52.0,M-SC,4.48,248.0,-0.29,1.17,21.79,XY24,NTT,AUTO
11,BANDHANBNK,1.98,-14.81,125.87,92.42,298691.0,-41258.0,237301.0,400.0,103.30,74.0,H-SC,6.49,174.0,-0.14,1.73,32.05,XY24,NTT,BANKS
83,WHIRLPOOL,-0.75,-26.43,147.28,81.92,124387.0,-30342.0,84456.0,2270.0,-56.86,59.0,M-SC,26.29,210.0,-0.24,0.61,18.64,XR,NTT,DURABLES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.68
1,25,45.50
2,50,77.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.00
MC    30.27
LC    24.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.99
X40      23.91
X40N     14.81
AR        9.02
XY25      8.90
XR        8.43
OX40N     7.10
SR        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.78
H-SC    22.09
X-LC    21.36
X-SC    11.52
M-SC    10.65
H-MC     5.02
M-MC     2.08
H-LC     1.20
L-LC     1.15
M-LC     1.01
L-SC     0.74
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.64,-14.64,53.42
FINANCE,13.11,-15.76,61.31
IT,11.20,-42.29,113.60
MISC,6.88,-37.19,93.31
ELECTRICAL,6.33,-7.86,47.22
PAINTS,5.06,-32.69,52.40
INSURANCE,4.83,-2.18,37.15
PHARMA,4.11,-3.93,36.64
AUTO,3.83,-18.36,63.49


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3415449.0,21
X40,1431588.0,15
XR,1331755.0,12
AR,1330707.0,10
X40N,1084786.0,10
OX40N,815635.0,10
XY25,393647.0,6
SR,304241.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3753231.0,24
M-SC,1453255.0,14
X-MC,1382184.0,15
X-LC,1332278.0,12
X-SC,1218734.0,10
H-MC,388577.0,3
L-SC,189388.0,2
M-LC,153008.0,1
M-MC,99984.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1271975.0      6
           AR         958110.0      5
           XR         857118.0      7
M-SC       XY24       841779.0      6
X-LC       X40        699641.0      6
X-SC       XY24       623926.0      4
X-MC       X40        548640.0      7
           X40N       439092.0      4
X-SC       X40N       411501.0      4
H-SC       OX40N      361787.0      4
M-SC       OX40N      357656.0      5
H-SC       SR         304241.0      2
X-LC       XY24       274149.0      2
           X40N       234193.0      2
H-MC       XY24       207606.0      1
X-MC       XY25       198438.0      2
           XY24       196014.0      2
X-SC       X40        183307.0      2
H-MC       AR         180971.0      2
M-SC       XR         178874.0      2
M-LC       XR         153008.0      1
X-LC       XY25       124295.0      2
L-SC       OX40N       96192.0      1
           XR          93196.0      1
M-SC       AR          74946.0      1
H-LC       AR          68387.0      1
M-MC       XY25        51691.0      1
L-MC       XR          49559.0      1
M-MC       AR          48293.0      1
L-LC       XY25        19223.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
